In [1]:
from __future__ import print_function, division
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from gensim import corpora, models, similarities, matutils

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import sklearn.metrics.pairwise as smp

/Users/KVASU/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
magic_cards = pd.read_csv("../MagicDatasets/Magic_Pandas_DF")

/Users/KVASU/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
magic_cards = magic_cards.drop(magic_cards.columns[0], axis=1)

In [22]:
magic_cards.iloc[:,22]

0                                               [JUD, 10E]
1             [PO2, S99, INV, 8ED, 9ED, 10E, DDC, DD3_DVD]
2                 [POR, PO2, EXO, S99, S00, 9ED, 10E, TPR]
3                                               [USG, 10E]
4                                [PO2, ODY, 10E, AVR, M14]
5                                    [WTH, pFNM, 10E, C15]
6                                     [ODY, 8ED, 9ED, 10E]
7                                          [MMQ, 9ED, 10E]
8                                          [STH, 10E, TPR]
9                                               [5DN, 10E]
10                                         [WTH, ATH, 10E]
11                                              [MMQ, 10E]
12                         [pMPR, DIS, 10E, M11, DDL, C14]
13                          [ONS, 8ED, 9ED, 10E, ROE, M12]
14                                              [CSP, 10E]
15                                              [GPT, 10E]
16                         [USG, pSUS, 7ED, 8ED, 9ED, 10

In [5]:
magic_cards_fill = magic_cards.fillna(" ")
magic_cards_fill['combined_text'] = magic_cards_fill['name']+" "+magic_cards_fill['type']+" "+magic_cards_fill['colors']+" "+magic_cards_fill["text"]+" "+magic_cards_fill["flavor"]+" "+magic_cards_fill["rarity"]
magic_cards_s = magic_cards_fill['combined_text']
magic_list = magic_cards_s.tolist
magic_list = list(magic_cards_s)

In [6]:
tfidf = TfidfVectorizer(stop_words="english", 
                        token_pattern="\\b[a-zA-Z0-9][a-zA-Z0-9]+\\b", 
                        min_df=10)


In [7]:
tfidf_vecs = tfidf.fit_transform(magic_list)


In [8]:
tfidif_df = pd.DataFrame(tfidf_vecs.todense(), 
             columns=tfidf.get_feature_names()
            )

In [9]:
new_cards = magic_cards
new_cards = new_cards.fillna(0.0)

magic_cards_fill_cut = new_cards.iloc[:,[2,11,21,34]]
magic_cards_fill_cut.info()

dummied_df = pd.get_dummies(magic_cards_fill_cut)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31705 entries, 0 to 31704
Data columns (total 4 columns):
cmc          31705 non-null float64
loyalty      31705 non-null float64
power        31705 non-null object
toughness    31705 non-null object
dtypes: float64(2), object(2)
memory usage: 990.9+ KB


In [10]:
combined_df = pd.concat([dummied_df, tfidif_df], axis=1, join_axes=[magic_cards_fill_cut.index])

In [11]:
model = NearestNeighbors(n_neighbors=20,n_jobs=-1)

In [12]:
knn = model.fit(combined_df)

/Users/KVASU/anaconda2/lib/python2.7/site-packages/pandas/indexes/base.py:1237: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return key in self._engine


In [25]:
# ATH, CN2, CNS, CON, CPK, CSP, CST, DDE, DDM, DDN, DDP, DDQ, DDR, DGM, DIS, DKM, DPA, DST, EMA, EMN, FRF_UGIN,
# GPT, GTC, H09, ITP, JUD, KLD, MBS, MD1, ME2, ME3, ME4, MED, MGB, MIR, MOR, MPS, NMS, ODY, OGW, PD3, PLS


set_dict = {
    "[10E]" : "Tenth Edition", "[2ED]" : "Unlimited", "[3ED]" : "Revised", "[4ED]" : "Fourth Edition",
    "[5DN]" : "Fifth Dawn", "[5ED]" : "Fifth Edition", "[6ED]" : "Sixth Edition", "[7ED]" : "Seventh Edition",
    "[8ED]" : "Eighth Edition", "[9ED]" : "Ninth Edition", "[ALA]" : "Shards of Alara", "[ALL]" : "Alliances",
    "[APC]" : "Apocalypse", "[ARB]" : "Alara Reborn", "[ARC]" : "Archenemy", "[ARN]" : "Arabian Nights",
    "[ATQ]" : "Aniquities", "[AVR]" : "Avacyn Restored", "[BFZ]" : "Battle for Zendikar", 
    "[BNG]" : "Born of the Gods", "[BOK]" : "Betrayers of Kamigawa", "[BRB]" : "Battle Royale", "[BTB]" : "Beatdown",              "[C13]" : "Commander 2013",
    "[C14]" : "Commander 2014", "[C15]" : "Commander 2015", "[CED]" : "Collectors' Edition",
    "[CEI]" : "International Edition", "[CHK]" : "Champions of Kamigawa", "[CHR]" : "Chronicles",
    "[CM1]" : "Commander's Arsenal", "[CMD]" : "Commander", "[C13]" : "Commander 2013",
    "[DD2]" : "Duel Decks: Jace vs. Chandra", "[DD3_DVD]" : "Duel Decks: Divine vs. Demonic", 
    "[DD3_EVG]" : "Duel Decks: Elves vs. Goblins", "[DD3_GVL]" : "Duel Decks: Garruk vs. Liliana",
    "[DD3_JVC]" : "Duel Decks: Jace vs. Chandra", "[DDC]" : "Duel Decks: Divine vs. Demonic",
    "[DDD]" : "Duel Decks: Garruk vs. Liliana", "[DDF]" : "Duel Decks: Elspeth vs. Tezzeret",
    "[DDG]" : "Duel Decks: Knights vs. Dragons", "[DDH]" : "Duel Deck: Ajani vs. Nicol Bolas",
    "[DDI]" : "Duel Decks: Venser vs. Koth", "[DDJ]" : "Duel Decks: Izzet vs. Golgari", 
    "[DDK]" : "Duel Decks: Sorin vs. Tibalt", "[DDL]" : "Duel Decks: Heroes vs. Monsters",
    "[DDO]" : "Duel Decks: Elspeth vs. Kiora", "[DKA]" : "Dark Ascension", "[FVD]" : "From the Vault: Dragons",
    "[DRK]" : "The Dark", "[DTK]" : "Dragons of Tarkir", "[EVE]" : "Eventide", "[EVG]" : "Duel Decks: Elves vs. Goblins",
    "[EXO]" : "Exodus", "[EXP]" : "Zendikar Expeditions", "[FEM]" : "Fallen Empires", "[FRF]" : "Fate Reforged",
    "[FUT]" : "Future Sight", "[HML]" : "Homelands", "[HOP]" : "Plancechase", "[ICE]" : "Ice Age", "[INV]" : "Invasion",
    "[ISD]" : "Innistrad", "[JOU]" : "Journey into Nyx", "[KTK]" : "Khans of Tarkir", "[LEA]" : "Alpha", 
    "[LEB]" : "Beta", "[LEG]" : "Legends", "[LGN]" : "Legions", "[LRW]" : "Lorwyn", "[M10]" : "Magic 2010", 
    "[M11]" : "Magic 2011", "[M12]" : "Magic 2012", "[M13]" : "Magic 2013", "[M14]" : "Magic 2014", 
    "[M15]" : "Magic 2015", "[MM2]" : "Modern Masters 2015", "[MMA]" : "Modern Masters", "[MMQ]" : "Mercadian Masques",
    "[MRD]" : "Mirrodin", "[NPH]" : "New Phyrexia", "[ONS]" : "Onslaught", "[ORI]" : "Magic Origins", 
    "[PC2]" : "Planarchase 2012", "[PCY]" : "Prophecy", "[KLD]" : "Kaladesh"
}

In [75]:
name = "Saheeli Rai"
card_index = (magic_cards_fill[magic_cards_fill['name']==name]).index.tolist()
card_index = card_index[0]

distances, indices = knn.kneighbors(combined_df.iloc[card_index,:])

index = indices[0]
distance = distances[0]

card_index

/Users/KVASU/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


15626

In [83]:
saheeli_df = magic_cards_fill[magic_cards_fill['name']==name]
saheeli_df
saheeli_df.iloc[:,22]
set_name = saheeli_df.iloc[:,22]
set_name
# set_name_list = set_name.split()
set_name[0]

KeyError: 0

In [49]:
set_name = list(magic_cards.iloc[card_index,22])

print (type(set_name))
print (set_name)
try:
    set_name = set_dict[magic_cards.iloc[card_index,22]]
except:
    set_name = magic_cards.iloc[card_index,22]

link = "http://www.cardkingdom.com/mtg/"+str(set_name)+"/"
print (link)

<type 'list'>
['[', 'A', 'L', 'A', ',', ' ', 'D', 'D', 'F', ',', ' ', 'M', 'M', 'A', ',', ' ', 'M', 'D', '1', ']']
http://www.cardkingdom.com/mtg/[ALA, DDF, MMA, MD1]/


In [24]:
nearest_list = []
i=0
for k in range(0,len(index)):
    card_name = magic_cards.iloc[index[k],16]
    if card_name not in nearest_list:
        if i<6:
            nearest_list.append(card_name)
            print (str(i) + ".")
            print (card_name)
            print ("Distance Away: "+str(distance[k]))
            print ("CMC: "+str(magic_cards.iloc[index[k],2])+" Power/Toughness: "+str(magic_cards.iloc[index[k],21])+"/"+str(magic_cards.iloc[index[k],21]))
            print ("Cost: "+str(magic_cards.iloc[index[k],12]))
            print ("Type: "+str(magic_cards.iloc[index[k],35]))
            print ("Sets: "+str(magic_cards.iloc[index[k],22]))
            print ("Text: "+str(magic_cards.iloc[index[k],32]))
            print ("Flavor: "+str(magic_cards.iloc[index[k],5]))
            print ("Link: ")+
            print ("------------")
            i+=1
        else:
            continue
    else:
        continue

print ("This is the chosen card and the 5 closest cards: "+str(nearest_list))

0.
Elspeth, Knight-Errant
Distance Away: 0.0
CMC: 4.0 Power/Toughness: nan/nan
Cost: {2}{W}{W}
Type: Planeswalker — Elspeth
Sets: [ALA, DDF, MMA, MD1]
Text: +1: Put a 1/1 white Soldier creature token onto the battlefield.
+1: Target creature gets +3/+3 and gains flying until end of turn.
−8: You get an emblem with "Artifacts, creatures, enchantments, and lands you control have indestructible."
Flavor: nan
------------
1.
Ajani Steadfast
Distance Away: 1.18545311408
CMC: 4.0 Power/Toughness: nan/nan
Cost: {3}{W}
Type: Planeswalker — Ajani
Sets: [pMEI, M15]
Text: +1: Until end of turn, up to one target creature gets +1/+1 and gains first strike, vigilance, and lifelink.
−2: Put a +1/+1 counter on each creature you control and a loyalty counter on each other planeswalker you control.
−7: You get an emblem with "If a source would deal damage to you or a planeswalker you control, prevent all but 1 of that damage."
Flavor: nan
------------
2.
Gideon, Ally of Zendikar
Distance Away: 1.2026888